In [1129]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
import numpy as np
import os
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
import pickle
import random

In [1130]:
class VGG_FeatureExtractor(nn.Module):
  def __init__(self):
    super(VGG_FeatureExtractor, self).__init__()
    model_vgg = models.vgg16(pretrained=True)
    self.features = model_vgg.features
    self.avgpool = model.avgpool
    self.classifier = model_vgg.classifier[0:3]

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), 25088)
    x = self.classifier(x)
    return x

In [1131]:
class office_loader(Dataset):
    def __init__(self, root= "/home/iacvr/dataset/", name = "office/amazon_10_list.txt", transform=None):
        self.path = root + name
        with open(self.path,"r") as file:
            self.data = [(Image.open(path.split()[0]).convert("RGB"), int(path.split()[1])) for path in file.readlines()]
        self.transform = transform
        file.close()
    
    def __getitem__(self, idx):
        img, label = self.data[idx][0], self.data[idx][1]
        if self.transform:
            img = self.transform(img)
        else:
            img = np.asarray(img).transpose((2, 0, 1))
            img = torch.tensor(np.asarray(img))
        return img, label
    
    def __len__(self):
        return len(self.data)

In [1132]:
class feature_dataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
            
    def __getitem__(self,idx):
        feature, label = self.features[idx], self.labels[idx]
        return torch.tensor(feature), label
    
    def __len__(self):
        return len(self.features)

In [1133]:
class ResizeImage():
    def __init__(self, size):
      if isinstance(size, int):
        self.size = (int(size), int(size))
      else:
        self.size = size
    def __call__(self, img):
      th, tw = self.size
      return img.resize((th, tw))

In [1134]:
def transform_data(resize_size=224, crop_size=224):
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  return  transforms.Compose([
        ResizeImage(resize_size),
        transforms.ToTensor(),
        normalize
    ])

In [1135]:
def get_features(G, data_loader):  # G is the feature extractor
    X = np.zeros((1,4096))
    Y = np.zeros((1), dtype= int)
    for images, labels in data_loader:
        features = G(images).data.numpy()
        labels = labels.numpy()
        X = np.concatenate((X,features), axis=0)
        Y = np.concatenate((Y,labels), axis=0)
    return X[1:], Y[1:]

In [1136]:
class HLoss(nn.Module):
    def __init__(self):
        super(HLoss, self).__init__()

    def forward(self, x):
        b = F.softmax(x, dim=1) * F.log_softmax(x, dim=1)
        b = -1.0 * b.sum(dim=1)
        return b.mean()

In [1220]:
class content_module(nn.Module):
    def __init__(self, feat_dim=4096, num_classes=31):
        super(content_module, self).__init__()
        self.feat_dim = feat_dim
        self.num_classes = num_classes
        self.fc1 = nn.Linear(self.feat_dim,int(self.feat_dim/4))
        self.fc2 = nn.Linear(int(self.feat_dim/4), self.num_classes)
    
    def forward(self, x):
        h_1 = F.relu(self.fc1(x))
        content = F.relu(self.fc2(h_1))
        return content

class style_module(nn.Module):
    def __init__(self, feat_dim=4096, style_dim = 100):
        super(style_module, self).__init__()
        self.feat_dim = feat_dim
        self.style_dim = style_dim
        self.fc1 = nn.Linear(self.feat_dim, int(self.feat_dim/4))
        self.fc2 = nn.Linear(int(self.feat_dim/4), self.style_dim)
    
    def forward(self, x):
        h_1 = F.relu(self.fc1(x))
        style = F.relu(self.fc2(h_1))
        return style

class reconstruction(nn.Module):
    def __init__(self, feat_dim=4096, num_classes = 31, style_dim = 100):
        super(reconstruction, self).__init__()
        self.feat_dim = feat_dim
        self.style_dim = style_dim
        self.num_classes = num_classes
        self.input_dim = style_dim + num_classes
        self.fc1 = nn.Linear(self.input_dim, int(self.feat_dim/4))
        self.fc2 = nn.Linear(int(self.feat_dim/4), self.feat_dim)
    
    def forward(self, content, style):
        m = nn.Softmax(dim=1)
        x = torch.cat((m(content), style), 1)
        h_1 = F.relu(self.fc1(x))
        h_2 = F.relu(self.fc2(h_1))
        return h_2

class adv_classifier_module(nn.Module):
    def __init__(self, num_classes=31, style_dim = 100):
        super(adv_classifier_module, self).__init__()
        self.style_dim = style_dim
        self.num_classes = num_classes
        self.fc1 = nn.Linear(self.style_dim, self.num_classes)
    
    def forward(self, x):
        logits = F.relu(self.fc1(x))
        return logits

class domain_identifier_module(nn.Module):
    def __init__(self, feat_dim = 4096):
        super(domain_identifier_module, self).__init__()
        self.feat_dim = feat_dim
        self.fc1 = nn.Linear(self.feat_dim, int(self.feat_dim/4))
        self.fc2 = nn.Linear(int(self.feat_dim/4), int(self.feat_dim/16))
        self.fc3 = nn.Linear(int(self.feat_dim/16), 2)
    
    def forward(self, x):
        h_1 = F.relu(self.fc1(x))
        h_2 = F.relu(self.fc2(h_1))
        h_3 = F.relu(self.fc3(h_2))
        return h_3

In [1379]:
class training_protocol():
    def __init__(self):
        pass
    
    def train_domain_identifier_module(self, data_loader, DI, epochs):
        print("Training domain identifier module")
        loss_ = nn.CrossEntropyLoss()
        optimizer = optim.SGD(DI.parameters(), lr = 0.003)
        epoch_loss = list()
        for epoch in range(epochs):
            batch_loss = list()
            for i, data in enumerate(data_loader, 0):
                inputs, labels = data[0].float(), data[1]
                logits = DI(inputs)
                optimizer.zero_grad()
                loss = loss_(logits, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                batch_loss.append(loss.item())

            if(epoch%5==0):
                print("epoch {} : Classification Loss {:2f}".format(epoch, np.mean(batch_loss)))
        print('Finished Training')
                
    def train_content_module(self, data_loader, C, epochs):
        print("Training Content Separator")
        loss_ = nn.CrossEntropyLoss()
        params = list(C.parameters())
        optimizer = optim.SGD(params, lr = 0.004)
        epoch_loss = list()
        for epoch in range(epochs):  # loop over the dataset multiple times
            batch_loss = list()
            for i, data in enumerate(data_loader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].float(), data[1]
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                content = C(inputs)
                loss = loss_(content, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                batch_loss.append(loss.item())

            if(epoch%5==0):
                print("epoch {} : Classification Loss {:2f}".format(epoch, np.mean(batch_loss)))
        print('Finished Training')
    
    def train_style_module(self, data_loader, C, S, R, adv_clf, epochs):
        print("Training Style Separator")
        loss_mse = nn.MSELoss()
        loss_cent = nn.CrossEntropyLoss()
        loss_ent = HLoss()
        params = list(S.parameters()) + list(R.parameters()) + list(adv_clf.parameters())
        optimizer = optim.Adam(params)
          
        epoch_adv_loss, epoch_rec_loss = list(), list()
        for epoch in range(epochs):  #loop over the dataset multiple times
            batch_adv_loss, batch_rec_loss, batch_ent_loss = list(), list(), list()
            for i, data in enumerate(data_loader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].float(), data[1]
                # forward + backward + optimize
                optimizer.zero_grad()
                content = C(inputs).detach()
                style = S(inputs)
                inputs_hat = R(content, style)
                logits = adv_clf(style)
                loss1 = loss_mse(inputs, inputs_hat)
                loss2 = -loss_ent((logits))
                loss = loss1 + loss2
                
                loss.backward()
                optimizer.step()
                batch_rec_loss.append(loss1.item())
                batch_adv_loss.append(loss2.item())
        
            if(epoch%5==0):
                print("epoch {} : Adversarial Loss {:2f} Reconstruction Loss {:2f}".format(epoch, np.mean(batch_adv_loss),np.mean(batch_rec_loss)))   
        print('Finished Training')
    
    def test_content(self, data_loader, C):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in data_loader:
                features, labels = data[0].float(), data[1]
                content = C(features)
                m = nn.Softmax(dim=1)
                outputs = m(content)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print("Accuracy of the network on the data: {:2f}".format(100 * (correct / total)))
    
    def test_style(self, data_loader, S, adv_clf):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in data_loader:
                features, labels = data[0].float(), data[1]
                style = S(features)
                logits = adv_clf(style)
                m = nn.Softmax(dim=1)
                outputs = m(logits)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print("Accuracy of the network on the data: {:2f}".format(100 * (correct / total)))
    
    def check_confidence(self, inputs, Ct, confidence, source_index_dict, source_features):
        x = Ct(inputs).detach()
        m = nn.Softmax(dim=1)
        x = m(x)
        values, indices = torch.max(x, 1)
        classes = list()
        source_inputs = torch.tensor([])
        target_inputs = torch.tensor([])
        for i in range(len(values)):
            if values[i]>=confidence:
                c = random.choice(source_index_dict[indices[i].item()])
                source_inputs = torch.cat((source_inputs, torch.tensor([source_features[c]]).float()), 0)
                target_inputs = torch.cat((target_inputs, inputs[i].unsqueeze(0)), 0)
        return source_inputs, target_inputs
    
    def train(self, source_index_dict, source_features, source_labels, target_loader, Cs, Ss, Rs, Ct, St, Rt, DI, epochs):
        print("Training")
        loss_ = nn.CrossEntropyLoss()
        loss_mse = nn.MSELoss()
        optimizer = optim.SGD([{"params":Ct.parameters(), "lr": 0.001},{"params":St.parameters(), "lr": 0.0},{"params":Rt.parameters(), "lr": 0.0},{"params":DI.parameters(), "lr": 0.0}])
        epoch_loss = list()
        for epoch in range(epochs):  # loop over the dataset multiple times
            batch_loss = list()
            for i, data in enumerate(target_loader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, _ = data[0].float(), data[1]
                source_inputs, target_inputs = self.check_confidence(inputs, Ct, 0.9, source_index_dict, source_features)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                if(len(source_inputs)==0):
                    continue
                source_content, source_style = Cs(source_inputs).detach(), Ss(source_inputs).detach()
                target_content, target_style = Ct(target_inputs), St(target_inputs)
                #X1, y1 = Rt(source_content, target_style), torch.tensor(np.zeros(len(source_inputs)), dtype = int)
                X2, y2 = Rt(target_content, source_style), torch.tensor(np.ones(len(source_inputs)), dtype = int)
                logits = DI(X2)
                loss = loss_(logits, y2)
                loss.backward()
                optimizer.step()
                # print statistics
                batch_loss.append(loss.item())

            if(epoch%5==0):
                print("epoch {} : Classification Loss {:2f}".format(epoch, np.mean(batch_loss)))
        print('Finished Training')

In [1380]:
# transform = transform_data()
# source = office_loader(name = "office/amazon_31_list.txt", transform = transform)
# target = office_loader(name = "office/webcam_31_list.txt", transform = transform)
# source_loader = DataLoader(source, batch_size = 64,shuffle = True)
# target_loader = DataLoader(target, batch_size = 64,shuffle = True)
# G = VGG_FeatureExtractor()
# source_features, source_labels = get_features(G, source_loader)
# target_features, target_labels = get_features(G, target_loader)

# features = np.concatenate([source_features, target_features], axis = 0)
# labels = np.concatenate([source_labels, target_labels], axis = 0)

# domain_features = np.concatenate([source_features, target_features], axis = 0)
# domain_source_features = source_features
# domain_target_features = target_features
# domain_labels = np.concatenate([np.ones(len(source_features), dtype = int), np.zeros(len(target_features), dtype = int)])
# domain_source_labels = np.ones(len(source_features), dtype = int)
# domain_target_labels = np.zeros(len(target_features), dtype = int)

# domain_features_train, domain_features_test, domain_labels_train, domain_labels_test = train_test_split(domain_features, domain_labels, test_size=0.2, random_state=42)

In [1381]:
# features_dataset = feature_dataset(features, labels)
# source_features_dataset = feature_dataset(source_features, source_labels)
# target_features_dataset = feature_dataset(target_features, target_labels)

# features_loader = DataLoader(features_dataset, batch_size = 32, shuffle = True)
# source_features_loader = DataLoader(source_features_dataset, batch_size = 32, shuffle = True)
# target_features_loader = DataLoader(target_features_dataset, batch_size = 32, shuffle = True)

# domain_features_train_dataset = feature_dataset(domain_features_train, domain_labels_train)
# domain_features_test_dataset = feature_dataset(domain_features_test, domain_labels_test)

# domain_features_train_loader = DataLoader(domain_features_train_dataset, batch_size = 32, shuffle = True)
# domain_features_test_loader = DataLoader(domain_features_test_dataset, batch_size = 32, shuffle = True)

# domain_source_features_dataset = feature_dataset(domain_source_features, domain_source_labels)
# domain_target_features_dataset = feature_dataset(domain_target_features, domain_target_labels)

# domain_source_features_loader = DataLoader(domain_source_features_dataset, batch_size = 32, shuffle = True)
# domain_target_features_loader = DataLoader(domain_target_features_dataset, batch_size = 32, shuffle = True)

# source_index_dict = {}
# target_index_dict = {}
# for i in range(31):
#     source_index_dict[i] = np.where(source_labels==i)[0]
#     target_index_dict[i] = np.where(target_labels==i)[0]

In [1382]:
# Cs = content_module()
# Ss = style_module()
# Rs = reconstruction()
# adv_clf = adv_classifier_module()

# Ct = content_module()
# St = style_module()
# Rt = reconstruction()

# DI = domain_identifier_module()

ti = training_protocol()

In [1384]:
# ti.train_content_module(data_loader = source_features_loader, C = Cs, epochs=100)
# ti.train_style_module(data_loader = features_loader, C = Cs, S = Ss, R = Rs, adv_clf = adv_clf, epochs=500)
# ti.train_domain_identifier_module(data_loader = domain_features_train_loader, DI = DI, epochs = 100)
# Ct.load_state_dict(Cs.state_dict())
# St.load_state_dict(Ss.state_dict())
# Rt.load_state_dict(Rs.state_dict())
# ti.train(source_index_dict, source_features, source_labels, target_features_loader, Cs, Ss, Rs, Ct, St, Rt, DI, 100)

In [1385]:
ti.test_content(data_loader = target_features_loader, C = Ct)
# ti.test_style(data_loader = source_features_loader, S = Ss, adv_clf = adv_clf)
# ti.test_content(data_loader = domain_source_features_loader, C = DI)

Accuracy of the network on the data: 61.132075


In [1386]:
content = Cs(torch.tensor(target_features).float())
style = Ss(torch.tensor(target_features).float())
features_hat = Rs(content, style).data.numpy()
print(features_hat.shape)
features_hat_dataset = feature_dataset(features_hat, domain_target_labels)
print(len(features_hat_dataset))
features_hat_loader = DataLoader(features_hat_dataset, batch_size = 32, shuffle = True)
ti.test_content(data_loader = features_hat_loader, C = DI)

(795, 4096)
795
Accuracy of the network on the data: 95.094340


In [1300]:
inputs = torch.tensor(target_features[0:794]).float()
labels = torch.tensor(target_labels[0:794])
x = Cs(inputs).detach()
m = nn.Softmax(dim=1)
x = m(x)
values, indices = torch.max(x, 1)
p = ((indices==labels))
q = (values>0.95)
labels_q = labels[q]
indices_q = indices[q]
print((labels_q==indices_q).sum().item()/q.sum().item())
print(q.sum().item()/len(target_features))
# classes = list()
# source_inputs = torch.tensor([])
# target_inputs = torch.tensor([])
# for i in range(len(values)):
#     if values[i]>=0.1:
#         c = random.choice(source_index_dict[indices[i].item()])
#         source_inputs = torch.cat((source_inputs, torch.tensor([source_features[c]]).float()), 0)
#         target_inputs = torch.cat((target_inputs, inputs[i].unsqueeze(0)), 0)
# print(source_inputs.shape)
# print(target_inputs.shape)
# print(len(source_inputs))

0.968503937007874
0.15974842767295597


### Rough Work

In [954]:
x = source_features_dataset[569][0].float()
content = Cs(x).detach().unsqueeze(0)
style = Ss(x).detach().unsqueeze(0)
x_hat = Rs(content, style).detach().squeeze()
print(x.norm(p=2))
print(x_hat.norm(p=2))
print(((x-x_hat).norm(p=2))**2)
print(F.mse_loss(x,x_hat, reduction='mean')*4096)
print(x)
print(x_hat)

tensor(83.2145)
tensor(73.4068)
tensor(1534.3530)
tensor(1534.3533)
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [856]:
model = models.vgg16(pretrained=True)
print(model.avgpool)

AdaptiveAvgPool2d(output_size=(7, 7))


In [228]:
import torch
import torch.nn as nn
import torch.optim as optim

n_dim = 5

p1 = nn.Linear(n_dim, 1)
p2 = nn.Linear(n_dim, 1)

optimizer = optim.Adam(list(p1.parameters())+list(p2.parameters()))
p2.weight.requires_grad = False
for i in range(4):
    dummy_loss = (p1(torch.rand(n_dim)) + p2(torch.rand(n_dim))).squeeze()
    optimizer.zero_grad()
    dummy_loss.backward()
    optimizer.step()
    print('p1: requires_grad =', p1.weight.requires_grad, ', gradient:', p1.weight.grad)
    print('p2: requires_grad =', p2.weight.requires_grad, ', gradient:', p2.weight.grad)
    print()

    if i == 1:
        p1.weight.requires_grad = False
        p2.weight.requires_grad = True

p1: requires_grad = True , gradient: tensor([[0.3403, 0.0636, 0.9248, 0.2385, 0.7509]])
p2: requires_grad = False , gradient: None

p1: requires_grad = True , gradient: tensor([[0.7751, 0.0997, 0.7676, 0.2848, 0.0819]])
p2: requires_grad = False , gradient: None

p1: requires_grad = False , gradient: tensor([[0., 0., 0., 0., 0.]])
p2: requires_grad = True , gradient: tensor([[0.0499, 0.1877, 0.8539, 0.5193, 0.5678]])

p1: requires_grad = False , gradient: tensor([[0., 0., 0., 0., 0.]])
p2: requires_grad = True , gradient: tensor([[0.2056, 0.0416, 0.5869, 0.0693, 0.1369]])



In [868]:
x = torch.tensor([[3.0,3.0,3.0],[3.0,3.0,3.0]])
y = torch.tensor([[1.0,1.0,1.0],[1.0,1.0,1.0]])

In [870]:
F.mse_loss(x,y, reduction ="sum")

tensor(24.)

In [955]:
a = np.array([10,15,26,15])
print(np.where(a==15))

(array([1, 3]),)
